# NN MVP or let's see

In [10]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

---

---

In [7]:
data = pd.read_csv("../../Source/Data/Train_data_n40.csv") 
data = data.drop(['Unnamed: 0'], axis=1)

In [8]:
data.shape

(312, 42)

In [11]:
X = data.iloc[:,2:].values
y = data['Labels']

In [14]:
X.shape, y.shape

((312, 40), (312,))

---

## Split data in train und test

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

In [16]:
# train data
X_train.shape, y_train.shape

((234, 40), (234,))

In [17]:
# test data
X_test.shape, y_test.shape

((78, 40), (78,))

-----

## Audio Recognition with LSTMs

In [18]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Zitat von https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html :

"Before getting to the example, note a few things. Pytorch’s LSTM expects all of its inputs to be 3D tensors. The semantics of the axes of these tensors is important. The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input." 

In [19]:
N = X_train.shape[0]

x = torch.tensor(X_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.long)

## Model

In [20]:
# RNN - LSTM
class Lstm(nn.Module):
    def __init__(self,  embeded_dim, hidden_dim):
        super(Lstm, self).__init__()
        self.embeded_dim = embeded_dim
        self.hidden_dim = hidden_dim

        self.inp = nn.Linear(16, self.embeded_dim)
        self.lstm = nn.LSTM(self.embeded_dim, self.hidden_dim)
        self.out = nn.Linear(self.hidden_dim, 10)

    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))
    
    def forward(self, cell_input):
        embedding = self.inp(cell_input)
        lstm_out, self.hidden = self.lstm(embedding, self.hidden)
        score = self.out(lstm_out)
        return score
    

### Hyperparameter

In [21]:
learning_rate = 0.001

n_epochs = 300
batch_size = 64

# Regularisierung
weight_decay=0.001

# the model
hidden_dim = 24
embeded_dim = 24
model = Lstm(embeded_dim, hidden_dim)

## Optimierer und Loss

https://pytorch.org/docs/stable/optim.html

In [22]:
# ADAM
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [23]:
# Loss
criterion = nn.CrossEntropyLoss()

## Train

In [24]:
loss_hist = []

In [34]:
# Train
epochs = range(n_epochs)
idx = 0
for t in epochs:
    for batch in range(0, int(N/batch_size)):
        # Step 1. Calculate Batch
        batch_x = x[batch * batch_size : (batch + 1) * batch_size, :]
        
        # dimensions: batch_size x 40 features?
        batch_x = batch_x.reshape( batch_size, 40).transpose(0,1)
        print(batch_x.shape)
        batch_y = y[batch * batch_size : (batch + 1) * batch_size]  
        print(batch_y.shape)
        
        # Step 2. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
            
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden(batch_size)
        
        # Step 3. Run our forward pass.
        output = model(batch_x)

        # Step 4. Berechne den Fehler mit dem letzten output 
        loss = criterion(output[40,:,:], batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Berechne den Fehler (Ausgabe des Fehlers alle 100 Iterationen)
    if t % 20 == 0:
        loss_hist.append(loss.item())
        print(t, loss.item())



torch.Size([40, 64])
torch.Size([64])


RuntimeError: size mismatch, m1: [40 x 64], m2: [16 x 24] at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/TH/generic/THTensorMath.cpp:2070

In [ ]:
plt.plot(loss_hist);

## Test

In [ ]:
x_test = torch.tensor(test_data, dtype=torch.float32)
y_test = torch.tensor(test_target, dtype=torch.long)

In [ ]:
x_test = x_test.reshape(-1, 16, 16).transpose(0,1)

In [ ]:
model.hidden = model.init_hidden(x_test.shape[1])
outputs = model(x_test)
y_pred = torch.max(outputs[15,:,:].data, 1)[1]

misclassifiction = 1.0 * (y_test != y_pred).sum().item()/y_pred.size()[0]

print('Misclassification Rate: ', 100 * misclassifiction, '%')